In [18]:
# Import necessary libraries
# scripts/functions.py
import sys
sys.path.append('../scripts')  # Appending the path to access the scripts folder
from functions4 import * 

In [19]:
# Full Task 4 Workflow
def satisfaction_analysis():
    # Step 1: Load Data
    df = load_data()

    df= clean_data(df)

    # Step 2: Define engagement and experience columns
    engagement_columns = ['Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)']  
    experience_columns = ['Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)'] 

    # Step 3: Scale the data
    scaled_engagement_data, scaled_experience_data = scale_data(df, engagement_columns, experience_columns)

    # Step 4: Perform KMeans clustering for engagement and experience
    df['Engagement_Cluster'], engagement_centroids = perform_kmeans_clustering(scaled_engagement_data, n_clusters=2)
    df['Experience_Cluster'], experience_centroids = perform_kmeans_clustering(scaled_experience_data, n_clusters=2)

    # Step 5: Calculate scores (assume cluster 0 for least engaged and worst experience)
    df = calculate_scores(df, scaled_engagement_data, scaled_experience_data, engagement_centroids[0], experience_centroids[1])

    # Step 6: Report top 10 satisfied customers
    top_10_customers = df.nlargest(10, 'Satisfaction_Score')
    print("Top 10 Satisfied Customers:")
    display(top_10_customers[[ 'Bearer Id', 'MSISDN/Number','Handset Type', 'Satisfaction_Score']])

    # Step 7: Train Regression Model
    
    model = train_regression_model(df)

    # Step 8: Run KMeans on scores
    df = kmeans_on_scores(df, n_clusters=2)

    # Step 9: Export results to PostgreSQL
    export_to_postgres(df)

    # Step 10: Deploy and track model using MLflow
    X_train = df[['Engagement_Score', 'Experience_Score']]
    y_train = df['Satisfaction_Score']
    deploy_model_with_tracking(model, X_train, y_train)




In [21]:


# Execute the workflow
satisfaction_analysis()


c:\Users\Toshiba\Documents\Kifiya\week 2\notebooks\../scripts\functions4.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Top 10 Satisfied Customers:


,Bearer Id,MSISDN/Number,Handset Type,Satisfaction_Score
42107,1.304243e+19,3.376206e+10,Huawei B528S-23A,56.086498
121741,1.304243e+19,3.366237e+10,Huawei B528S-23A,46.951569
36996,7.277826e+18,3.365850e+10,Huawei B528S-23A,46.822431
59412,7.277826e+18,3.365850e+10,Huawei B528S-23A,46.748967
76446,7.277826e+18,3.366512e+10,Huawei B528S-23A,46.723010
36903,1.304243e+19,3.366393e+10,Huawei B528S-23A,46.716135
985,7.277826e+18,3.376375e+10,Huawei B528S-23A,46.716120
142424,1.304243e+19,3.366411e+10,Huawei B528S-23A,46.678040
78608,1.304243e+19,3.369858e+10,Huawei B528S-23A,46.629860
145449,1.304243e+19,3.369991e+10,Huawei P20 Lite Huawei Nova 3E,46.582825


Mean Squared Error: 0.11390105418906057


2024/09/09 16:13:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
